<a href="https://colab.research.google.com/github/SeongMinKin/preproject/blob/master/200820.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow를 활용한 네이버쇼핑의 상품 카테고리 자동 분류

1. 카테고리 자동 매칭의 필요성

네이버쇼핑에 등록된 업체는 상품 등록을 위해 상품 정보를 네이버 쇼핑으로 보냄. 그러나 업체마다 다른 카테고리 정보를 가지고 있어 이를 네이버쇼핑에 맞게 매칭하는 것이 필수, but 네이버 쇼핑에 등록된 상품의 개수는 약 15억개, 하루 평균 2000만개 새로운 상품 등록이 이루어짐 
네이버에서 관리하는 카테고리 수는 5000개.

-> 이를 딥러닝의 분류 문제로 보고 해결하고자 한다.

2. 카테고리 자동 매칭 모델

1) Find Product's feature: 상품 정보에서 유용한 feature를 찾는다.

상품명만으로는 카테고리를 구분할 수 없는 경우가 생기므로 이미지를 같이 주요 feature로 사용한다.
또한, 네이버쇼핑 카테고리와는 다른 업체 고유의 카테고리, 상품의 최저가, 브랜드 메이커 등을 추가 feature로 활용했다.

2) Morphology Analysis: 상품 정보에서 term을 분석하고 추출한다.

주요 feature로 사용될 상품명에는 다음과 같은 특징이 있다.

-상품명에 주로 명사가 사용된다.
-한국어와 영어가 혼용된 상품명이 있다 등

상품명에 있는 단어가 올바른 의미를 가지려면 띄어쓰기가 없는 상품명은 term 단위로 추출되면서도 콘셉트를 설명하는
단어나 모델 코드, 상품 속성 등은 분리되지 않아야 한다.
네이버 사내 언어처리시스템을 이용하여 term을 추출한 뒤 특수문자는 제거하였다.

3) Word Embedding: 상품의 feature를 vector로 변환한다.

단어 사전을 만든 뒤 숫자를 부여하는 random sequence 방식은 단어의 의미와 관계를 잃기 쉬우므로 단어를 고차원에
매핑하여 vector로 표현하는 방식을 사용했다. (Word2vec)방법 사용, 상품 데이터의 특징이 포함된 word embedding구축했다.

4) CNN-LSTM Model - Product Name: 상품명에 CNN-LSTM 모델을 적용한다.

	1.CNN모델 : 상품명에서 텍스트 특정 지역의 feature추출
	이미지뿐 아니라 자연어처리에서도 CNN알고리즘을 적용하려는 많은 노력이 있었다.
	네이버쇼핑의 상품명에서는 위치와 무관하게 나타나는 상품의 주요 키워드를 볼 수 있다.
	이미지 처리에서 CNN 필터는 이미지 특정 지역의 feature를 추출할 수 있고, 
	텍스트 처리에서는 텍스트 특정 지역의 feature를 추출할 수 있다.

	2.LSTM모델 : 길이가 긴 상품명에서 주변 단어를 기반으로 현재 단어의 의미 파악
	RNN의 긴기간 의존성 문제를 해결한 모델이며 단어의 의미를 주변 단어들을 기반으로 파악
	->글과 같이 순차적으로 등장하는 데이터 처리에 적합
	네이버쇼핑의 상품명에서는 주변의 키워드를 함께 보아야 분류가 가능한 상품명을 볼 수 있다.

	3.CNN-LSTM모델 : CNN으로 추출한 지역적인 feature를 LSTM에서 순차적으로 통합해 사용한다.
	CNN-LSTM 모델은 일반적인 Neural Network모델보다 훨씬 더 뛰어난 성능을 모인다. word embedding을 입력으로
	컨볼루션 레이어와 max 풀링 레이어를 통과해 지역적 특징을 추출한 다음 LSTM모델을 사용해 feature를 연속으로
	통합하는 방식이다.
	CNN 모델은 입력된 텍스트를 여러 영역으로 나누어 특정 지역 정보를 추출할 수 있으나 긴 문장에서 단어간 의존성을
	파악하지 모사는데, 이를 해결한 모델이다.

5) MobileNetV2 - Product Image: 상품 이미지에 MobileNetV2 모델을 적용한다.

MobileNet은 상대적으로 적은 리소스로 이미지분류, 개체 탐지가 가능한 모델이다.
기존모델인 VGG와 비겨했을 때 MobileNet의 정확도는 비슷하지만 연산량과 파라미터 개수가 1/10수준이다.

6) Multi Input Model – Product Misc. Information: 상품이 가지고 있는 다양한 데이터를 입력으로 
모델을 연결해 정확도를 높인다.

상품명을 사용한 모델과 이미지를 사용한 모델을 별개로 학습시키는 것이 아닌 (관련된 데이터이므로) 함께 학습하면
모델의 정확도가 향상된다. 상품명과 이미지뿐만 아니라 다음 그림과 같이 상품이 부가적으로 가지고 있는 브랜드, 메이커, 
원본 이미지 비율, 업체에서 사용하는 카테고리, 상품의 최저가까지 함께 학습하면 상품에 관련된 속성의 관계를 학습할 수 있다.

3. 카테고리 자동 매칭에서 발견된 문제와 해결 방법

1) Feature Visualization
카테고리 자동매칭모델을 적용하기 전 카테고리 매칭 결과가 올바른 분포를 가지고 있는지 확인할 수 있는 방법이 있어야 한다.
TensorFlow의 Embedding Projector에 데이터에 맞는 feature vector와 label을 업로드하면 카테고리 분포와 카테고리 간의 
거리를 눈으로 쉽게 확인할 수 있다 4000개의 카테고리 vector를 시각화했을 때 서로 연관이 없는 카테고리가 가까이 위치해
있다면 매칭이 잘못될 가능성이 높은 카테고리라고 할 수 있다. (예전에 자연어처리 수업에서 한 글자 벡터화해서 시각화해본거에요)

2) Data Normalization
데이터의 불균형하다면 상품 수가 더 많은 카테고리로 학습이 편향될 수 있다. Data Normaliztion을 적용해 카테고리별 상품을
고르게 분포시켜 이 문제를 해결한다.

3) Reflecting trends
온라인 쇼핑몰의 상품은 수명이 짧고 유행에 민감하다. 따라서 패션 의류같은 경우는 계절별로 상품이 삭제되었다 생성되었다를
반복하는 현상을 볼 수 있다. 이러한 경우에 다음 그래프와 같이 Long-Term Accuracy를 분석해 모델이 앞으로도 정확하게
예측할 수 있는지 확인하고 학습 데이터와 메드 또한 그에 맞춰 지속적으로 업데이트하면서 상품의 유행을 따라갈 수 있어야 한다.

4) Training Data Pipeline
시간과 공간의 효율성을 위해 tensorflow의 pipeline을 이용한다.

4. 남아있는 작업

1) 분류가 모호한 카테고리
네이버쇼핑의 카테고리는 사용자 편의를 위한 카테고리 구조로, 동일한 이름의 하위 카테고리가 다수 존재해 상위 카테고리를
구별하기 어렵다. 모델이 학습하기에는 좋지않기에 별도의 학습이 필요한 카테고리를 선별해 보델이 학습하기 쉬운 구조로
변경하거나, 별도로 관리해 학습하는 방식이 필요하다.

2) 업체에서 유입되는 오류 정보
업체에서 카테고리를 보내주었을 때 오류가 내포된 경우도 있다. 이를 해결하기 위해 학습데이터에서 오류 데이터를 제거하기 
위해 품질 높은 상품 정보를 보내는 업체를 선정해 학습에 가중치를 부여하는 방법을 사용할 수 있다.

5. 마치며
현재 카테고리 자동 매칭의 정확도는 약 85%이다. 상품명 데이터 정제, 이미지 feature 추출 효율화, 정확한 검증 세트 구축 등의
다양한 방법을 활용해 모델이 더욱 높은 정확도를 가질 수 있도록 개선해 나가고자 한다